# SYMFLUENCE Tutorial 02b-troute — Semi-Distributed Routing with t-route (NOAA OWP)

## Introduction

This notebook is a variant of Tutorial 02b that substitutes **t-route** (NOAA's Office of Water Prediction channel routing model) for mizuRoute. The underlying hydrological model (SUMMA) and domain setup are identical.

**t-route** supports:
- Muskingum-Cunge and diffusive wave routing methods
- Integration with NWM (National Water Model) conventions
- Python-native execution (no compiled Fortran binary needed)

We reuse the **Bow River at Banff** semi-distributed domain from Tutorial 02b, switching only the routing model configuration.

## Step 1 — Configuration

We use the same domain as 02b (`Bow_at_Banff_semi_distributed`) but configure t-route as the routing model with a separate experiment ID.

In [ ]:
# Step 1 — Semi-distributed configuration with t-route routing

from pathlib import Path
import shutil
from symfluence import SYMFLUENCE
from symfluence.core.config.models import SymfluenceConfig

config = SymfluenceConfig.from_minimal(
    # Basic identification (reuse same domain)
    domain_name='Bow_at_Banff_semi_distributed',
    experiment_id='run_troute',

    # Gauging station coordinates (Banff WSC 05BB001)
    pour_point_coords='51.1722/-115.5717',

    # Semi-distributed basin settings
    definition_method='delineate',
    DELINEATION_METHOD='stream_threshold',
    STREAM_THRESHOLD=5000,
    discretization='GRUs',

    # Model configuration — SUMMA + t-route
    model='SUMMA',
    routing_model='troute',
    TROUTE_FROM_MODEL='SUMMA',

    # t-route routing configuration
    TROUTE_ROUTING_METHOD='muskingum_cunge',
    SETTINGS_TROUTE_DT_SECONDS=3600,
    TROUTE_MANNINGS_N=0.035,

    # SUMMA still needs its routing var for runoff extraction
    SETTINGS_MIZU_ROUTING_VAR='averageRoutedRunoff',

    # Temporal extent (same as 02b)
    time_start='2004-01-01 01:00',
    time_end='2007-12-31 23:00',
    calibration_period='2005-10-01, 2006-09-30',
    evaluation_period='2006-10-01, 2007-12-30',
    spinup_period='2004-01-01, 2005-09-30',

    # Streamflow observations
    station_id='05BB001',
    DOWNLOAD_WSC_DATA=True,
)

# Save configuration
import yaml
config_path = Path('./config_semi_distributed_troute.yaml')
config_dict = config.to_dict(flatten=True)
with open(config_path, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False, sort_keys=False)
print(f'Configuration saved: {config_path}')

# Verify routing model
print(f'Routing model: {config_dict.get("ROUTING_MODEL")}')
print(f'Routing method: {config_dict.get("TROUTE_ROUTING_METHOD")}')
print(f'Manning\'s n: {config_dict.get("TROUTE_MANNINGS_N")}')
print(f'dt (seconds): {config_dict.get("SETTINGS_TROUTE_DT_SECONDS")}')

In [ ]:
# Initialize SYMFLUENCE
symfluence = SYMFLUENCE(config, visualize=True)
project_dir = symfluence.managers['project'].setup_project()
print(f'Project directory: {project_dir}')

## Step 2 — Domain definition

If the domain was already delineated in Tutorial 02b, the shapefiles and GRU discretization will be reused automatically. Otherwise, run delineation and discretization.

In [ ]:
# Step 2 — Domain definition (reuse from 02b if available)
shp_dir = project_dir / 'shapefiles' / 'river_network'
if shp_dir.exists() and len(list(shp_dir.glob('*.shp'))) > 0:
    print('Stream network shapefiles found — reusing existing delineation')
else:
    print('Delineating stream network...')
    symfluence.managers['domain'].define_domain()
    print('Delineation complete')

hru_dir = project_dir / 'shapefiles' / 'catchment'
if hru_dir.exists() and len(list(hru_dir.glob('*.shp'))) > 0:
    print('GRU discretization found — reusing')
else:
    print('Discretizing domain...')
    symfluence.managers['domain'].discretize_domain()
    print('Discretization complete')

## Step 3 — Model preprocessing (SUMMA + t-route)

This step creates:
1. SUMMA configuration files (forcings, parameters, file manager)
2. **t-route topology file** (NetCDF with NWM-style variable names: `comid`, `to_node`, `link`, `nhru`)
3. **t-route YAML configuration** (`troute_config.yml` with forcing paths and timestep settings)

In [ ]:
# Step 3a — Model-agnostic preprocessing (forcing remapping, etc.)
symfluence.managers['data'].run_model_agnostic_preprocessing()
print('Model-agnostic preprocessing complete')

In [ ]:
# Step 3b — Model-specific preprocessing (SUMMA + t-route)
symfluence.managers['model'].preprocess_models()

# Verify t-route settings were created
troute_settings = project_dir / 'settings' / 'troute'
if troute_settings.exists():
    print(f'\nt-route settings directory: {troute_settings}')
    for f in sorted(troute_settings.iterdir()):
        print(f'   {f.name} ({f.stat().st_size:,} bytes)')
else:
    print(f'WARNING: t-route settings not found at {troute_settings}')

print('\nModel preprocessing complete')

## Step 4 — Model execution (SUMMA + t-route)

SUMMA runs first to generate HRU-level runoff, then t-route routes it through the stream network.

t-route executes as a Python module (`python -m nwm_routing`) rather than a compiled binary, and reads its input from the SUMMA output NetCDF file with the runoff variable renamed to `q_lateral`.

In [ ]:
# Step 4 — Run SUMMA + t-route
symfluence.managers['model'].run_models()
print('SUMMA + t-route simulation complete')

## Step 5 — Evaluation

Compare t-route routed streamflow against observations at the Banff gauge (WSC 05BB001).

In [ ]:
# Step 5a — Model comparison overview
from IPython.display import Image, display

plot_path = symfluence.managers['reporting'].generate_model_comparison_overview(
    experiment_id='run_troute',
    context='run_model'
)

if plot_path:
    print(f'Model comparison overview: {plot_path}')
    display(Image(filename=str(plot_path)))
else:
    print('No model outputs found for comparison. Check simulation outputs.')

print('\nEvaluation complete')

In [ ]:
# Step 5b — t-route specific routing diagnostics
from symfluence.models.troute.plotter import TRoutePlotter
import logging

plotter = TRoutePlotter(config_dict, logging.getLogger('troute_plotter'))
routing_plot = plotter.plot(experiment_id='run_troute')

if routing_plot:
    print(f't-route diagnostics: {routing_plot}')
    display(Image(filename=str(routing_plot)))
else:
    print('t-route diagnostics not generated (output data may not be available yet)')

## Step 6 — Compare with mizuRoute (optional)

If Tutorial 02b has already been run, we can load both routing results and compare.

In [ ]:
# Step 6 — Side-by-side routing comparison (optional)
import xarray as xr
import pandas as pd
import numpy as np

sim_base = project_dir / 'simulations'
mizu_dir = sim_base / 'run_1' / 'mizuRoute'
troute_dir = sim_base / 'run_troute' / 'TRoute'

def load_routing_outlet(sim_dir, router_name):
    """Load routed discharge at outlet from routing output."""
    nc_files = sorted(sim_dir.glob('*.nc'))
    if not nc_files:
        print(f'No output files found for {router_name} in {sim_dir}')
        return None
    ds = xr.open_dataset(nc_files[-1])
    # Try common flow variable names
    for var in ['IRFroutedRunoff', 'flow', 'streamflow', 'q_lateral', 'discharge']:
        if var in ds:
            data = ds[var]
            # Find spatial dim and select outlet (max mean flow)
            for dim in ['seg', 'reachID', 'feature_id', 'link']:
                if dim in data.dims:
                    outlet_idx = int(data.mean('time').argmax())
                    data = data.isel({dim: outlet_idx})
                    break
            series = data.to_pandas()
            series.name = router_name
            ds.close()
            return series
    ds.close()
    print(f'No flow variable found for {router_name}')
    return None

mizu_q = load_routing_outlet(mizu_dir, 'mizuRoute') if mizu_dir.exists() else None
troute_q = load_routing_outlet(troute_dir, 't-route') if troute_dir.exists() else None

if mizu_q is not None and troute_q is not None:
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(12, 4))
    ax.plot(mizu_q.index, mizu_q.values, label='mizuRoute', alpha=0.8)
    ax.plot(troute_q.index, troute_q.values, label='t-route', alpha=0.8, linestyle='--')
    ax.set_ylabel('Discharge (m\u00b3/s)')
    ax.set_title('Routing Model Comparison: mizuRoute vs t-route (Bow at Banff)')
    ax.legend()
    ax.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Basic stats
    common = mizu_q.index.intersection(troute_q.index)
    if len(common) > 0:
        corr = np.corrcoef(mizu_q.loc[common], troute_q.loc[common])[0, 1]
        rmse = np.sqrt(np.mean((mizu_q.loc[common] - troute_q.loc[common])**2))
        print(f'\nCorrelation: {corr:.4f}')
        print(f'RMSE: {rmse:.3f} m\u00b3/s')
        print(f'mizuRoute mean: {mizu_q.loc[common].mean():.2f} m\u00b3/s')
        print(f't-route mean: {troute_q.loc[common].mean():.2f} m\u00b3/s')
elif mizu_q is None:
    print('mizuRoute outputs not found. Run Tutorial 02b first for comparison.')
elif troute_q is None:
    print('t-route outputs not found. Run Step 4 above first.')